## <center>Working with Excel files and SQLite Database</center>

## Part 1

We need to read the nine stock files and insert their data into the database. 

The file names all start with a year, from 2009 straight to 2019 with no breaks:

```
2009_aapl_data.xlsx
2010_aapl_data.xlsx
2011_aapl_data.xlsx
2012_aapl_data.xlsx
2013_aapl_data.xlsx
2014_aapl_data.xlsx
2015_aapl_data.xlsx
2016_aapl_data.xlsx
2017_aapl_data.xlsx
2018_aapl_data.xlsx
2019_aapl_data.xlsx
```

In [54]:
import dataset
from openpyxl import load_workbook

In [55]:
input_name = '_aapl_data.xlsx'
db_file = 'stock_prices.db'

In [56]:
db = dataset.connect("sqlite:///" + db_file)

In [57]:
db.tables

['aapl']

In [58]:
if (len(db.tables) > 0):
    for table in db.tables:
        db[table].drop()

In [59]:
table = db.create_table("aapl")

In [60]:
db.tables

['aapl']

In [61]:
header = sheet[1]
header[0].value     # This is just to sanity-check that we got the header

'date'

In [62]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

def get_type(value):
    if value.isdigit():
        return dataset.types.Integer
    elif isfloat(value):
        return dataset.types.Float
    elif '/' in value:
        return dataset.types.Date
    else:
        return dataset.types.Unicode

In [63]:
row2 = sheet[2]

for cell in row2:
    print(f'{cell.value} is {get_type(cell.value)}')

2019/09/04 is <class 'sqlalchemy.sql.sqltypes.Date'>
209.1900 is <class 'sqlalchemy.sql.sqltypes.Float'>
19216820.0000 is <class 'sqlalchemy.sql.sqltypes.Float'>
208.3900 is <class 'sqlalchemy.sql.sqltypes.Float'>
209.4800 is <class 'sqlalchemy.sql.sqltypes.Float'>
207.3200 is <class 'sqlalchemy.sql.sqltypes.Float'>


In [64]:
for index, col_name in enumerate(header):
    table.create_column(col_name.value, get_type(row2[index].value))
db['aapl'].columns

['id', 'date', 'close', 'volume', 'open', 'high', 'low']

In [65]:
from datetime import datetime,date
keys = table.columns[1:]

for i in range(2009, 2020):
    data_file = "data/" + str(i) + input_name
    workbook = load_workbook(filename=data_file)
    sheet = workbook.active
    print(i)
    
    for values in list(sheet.values)[1:]:
        row = []
        row.append(datetime.strptime(values[0],'%Y/%m/%d').date())
        row = row + list(values[1:])
        d_row = dict(zip(keys,row))
        table.insert(d_row)


2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


In [66]:
# count rows in the table
print(f'Rows in database: {len(table)}')

Rows in database: 2516


## Part 2

Now that I have a working database with a reasonable amount of data in it, I am going to do some queries with it and show the data:

1. Find all days where the stock closed lower than 25. 
    * Print a count of how many
    * Print the first 5 rows found
2. Find all days in 2017 where the stock closed above 35.
    * Print a count of how many
    * Print the last 5 found.
3. Create a new workbook and put each query result on a new worksheet in the workbook. 


In [67]:
def print_rows():
    for row in sheet.iter_rows(values_only=True):
        print(row)

In [68]:
from openpyxl import Workbook
from openpyxl.chart import BarChart, Reference

In [69]:
import dataset
db = dataset.connect("sqlite:///stock_prices.db")
print(db.tables)

['aapl']


In [70]:
for i, row in enumerate(db['aapl']):
    if i < 5:
        print(row)

OrderedDict([('id', 1), ('date', datetime.date(2009, 12, 31)), ('close', 30.1046), ('volume', 87907426.0), ('open', 30.4471), ('high', 30.4786), ('low', 30.08)])
OrderedDict([('id', 2), ('date', datetime.date(2009, 12, 30)), ('close', 30.2343), ('volume', 102705781.0), ('open', 29.8328), ('high', 30.2857), ('low', 29.7586)])
OrderedDict([('id', 3), ('date', datetime.date(2009, 12, 29)), ('close', 29.8714), ('volume', 110755363.0), ('open', 30.3757), ('high', 30.3886), ('low', 29.8186)])
OrderedDict([('id', 4), ('date', datetime.date(2009, 12, 28)), ('close', 30.23), ('volume', 160784168.0), ('open', 30.2457), ('high', 30.5643), ('low', 29.9444)])
OrderedDict([('id', 5), ('date', datetime.date(2009, 12, 24)), ('close', 29.8628), ('volume', 125222058.0), ('open', 29.0786), ('high', 29.9071), ('low', 29.05)])


In [71]:
# Showing the stock closed lower than 25 which is 6 days in 2009
high_close = db['aapl'].find(close = {'<=': 25})
for row in high_close:
    vals = [v for k, v in row.items()]
    print(vals)

[77, datetime.date(2009, 9, 14), 24.8171, 80383404.0, 24.4043, 24.8428, 24.3214]
[78, datetime.date(2009, 9, 11), 24.5943, 87108026.0, 24.7014, 24.74, 24.41]
[79, datetime.date(2009, 9, 10), 24.6514, 122612107.0, 24.58, 24.75, 24.4014]
[80, datetime.date(2009, 9, 9), 24.4486, 202624511.0, 24.6828, 24.9243, 24.2428]
[81, datetime.date(2009, 9, 8), 24.7043, 78524974.0, 24.7114, 24.7343, 24.5714]
[82, datetime.date(2009, 9, 4), 24.33, 93309888.0, 23.8966, 24.3857, 23.87]


In [72]:
# Find all days where the stock closed lower than 25.

high_close = db['aapl'].find(close = {'<=': 25})
excel_rows = [] 

for row in high_close:
    vals = [v for k, v in row.items()]
    excel_rows.append(vals[1:])
    
# Print the first 5 rows found
excel_rows[:5]

[[datetime.date(2009, 9, 14), 24.8171, 80383404.0, 24.4043, 24.8428, 24.3214],
 [datetime.date(2009, 9, 11), 24.5943, 87108026.0, 24.7014, 24.74, 24.41],
 [datetime.date(2009, 9, 10), 24.6514, 122612107.0, 24.58, 24.75, 24.4014],
 [datetime.date(2009, 9, 9), 24.4486, 202624511.0, 24.6828, 24.9243, 24.2428],
 [datetime.date(2009, 9, 8), 24.7043, 78524974.0, 24.7114, 24.7343, 24.5714]]

In [73]:
# Print a count of how many days where the stock closed lower than 25. Output:6
len(excel_rows)

6

In [74]:
#Find all days in 2017 where the stock closed above 35.

high_close = db['aapl'].find(close = {'>=': 35}, date = {'between': ['2017-01-01', '2017-12-31']})
excel_rows = [] 

for row in high_close:
    vals = [v for k, v in row.items()]
    excel_rows.append(vals[1:])
    
#Print the last 5 found. 
excel_rows[-5:]

[[datetime.date(2017, 1, 9), 118.99, 33420290.0, 117.95, 119.43, 117.94],
 [datetime.date(2017, 1, 6), 117.91, 31714820.0, 116.78, 118.16, 116.47],
 [datetime.date(2017, 1, 5), 116.61, 22115100.0, 115.92, 116.8642, 115.81],
 [datetime.date(2017, 1, 4), 116.02, 21081620.0, 115.85, 116.51, 115.75],
 [datetime.date(2017, 1, 3), 116.15, 28769220.0, 115.8, 116.33, 114.76]]

In [75]:
#Print a count of how many days where the stock closed above 35. Output:251
len(excel_rows)

251

In [76]:
#create workbook and save the file 
workbook = Workbook() # Create a new workbook
sheet = workbook.active # Get the active worksheet

header = db['aapl'].columns[1:]

# converting to string
header = [str(v) for v in header]
sheet.append(header)
for row in excel_rows:
    sheet.append(row)

In [78]:
#saving the workbook
fname = "workbook_queries.xlsx"
workbook.save(filename=fname)